In [1]:
import sys
sys.path.append("../../")
from radarange_orchestrator import llm

In [2]:
from radarange_orchestrator.tools import pdf_tool, ls_tool
m = llm()
default_tools = [pdf_tool, ls_tool]

llama_init_from_model: n_ctx_per_seq (80128) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [3]:
from pydantic import BaseModel, RootModel
from llama_cpp import LlamaGrammar
import json

class Article(BaseModel):
    """Reference information, including authors, title, journal and date"""
    authors: str
    title: str
    journal: str
    issue_number_page: str | None = None
    year: int | None = None

class ArticleList(RootModel[list[Article]]):
    """References list of a science article"""
    root: list[Article]

schema_dict = ArticleList.model_json_schema()
print(schema_dict)
grammar = LlamaGrammar.from_json_schema(json.dumps(schema_dict))

{'$defs': {'Article': {'description': 'Reference information, including authors, title, journal and date', 'properties': {'authors': {'title': 'Authors', 'type': 'string'}, 'title': {'title': 'Title', 'type': 'string'}, 'journal': {'title': 'Journal', 'type': 'string'}, 'issue_number_page': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Issue Number Page'}, 'year': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Year'}}, 'required': ['authors', 'title', 'journal'], 'title': 'Article', 'type': 'object'}}, 'description': 'References list of a science article', 'items': {'$ref': '#/$defs/Article'}, 'title': 'ArticleList', 'type': 'array'}


In [4]:
from radarange_orchestrator.chat import Chat
from radarange_orchestrator.tools.grammar_switch_tool import create_set_grammar_tool


path = "downloads/Drofa_NV_centers_2023.pdf"
prompt = """
В папке downloads лежит статья в формате pdf. Заполни информацию о ссылках из неё.
"""
def attempt(system_prompt: str) -> tuple[str, Chat]:
    chat = m.chat(tools = default_tools)
    chat.add_user_message(prompt)
    chat.add_system_message(system_prompt)
    
    grammar_context, fly_tool = create_set_grammar_tool(grammar)
    chat.tools.append(fly_tool)
    r, inner_chat = m.act_with_grammar(
        chat, grammar_context=grammar_context,
        max_tokens_per_message=10000, temperature=0, max_prediction_rounds=10
    )
    return r.content, inner_chat

In [ ]:
from radarange_orchestrator.llm_types import Response

def prepare_prompt(prompt: str, inner_chat: Chat, history: list[str]) -> str:
    llm_messages: list[str] = []
    for message in inner_chat.history:
        if isinstance(message, Response):
            llm_messages.append(message.content)

    joined_history = '\n'.join([f'[Attempt {i}]: {text}' for i, text in enumerate(history)])
    pre_prompt = f"""
You need to adjust the system prompt so that llm would do a sequence of acts, and the final response should apply to given json formatting
Here is the user prompt that will be passed to llm to fulfill:
{prompt}

Here is the first message from llm from previous acting sequence:
{llm_messages[0]}

Here is the last message, that should've been the right json format, but failed validation:
{llm_messages[-1]}

The history of system prompt attempts:
{joined_history}
    
Now you need to print only the next system prompt as in previous system prompt examples, and nothing else.
"""
    return pre_prompt

In [6]:
from pydantic import ValidationError

def validate(r: str) -> bool:
    try:
        ArticleList.model_validate_strings(r)
    except ValidationError:
        return False
    return True

In [ ]:
system_prompt = m.DEFAULT_GRAMMAR_SYSTEM_PROMPT
history = [system_prompt]
inner_chats = []

N = 2
for i in range(N):
    r, inner_chat = attempt(history[-1])
    inner_chats.append(inner_chat)
    if validate(r):
        break
    
    pre_prompt = prepare_prompt(prompt, inner_chat, history)
    r = m.respond(pre_prompt, temperature=0, max_tokens = 2000)
    history.append(r.content)

Called list_directory with path: downloads
Called pdf_read with path: downloads/Drofa_NV_centers_2023.pdf
Taken 0.2 seconds to complete.
Called switch grammar


In [8]:
history

['\nUser expects you to fulfill the task in a few separate steps.\nSince output is strictly formatted, but perhaps you need to make some intermediate generations, grammar is yet unset.\nBefore telling the answer to user, call grammar switch tool and wait for it to successfully complete.\nThe next response from you will be applied with correct grammar, so make sure to separate output into separate message.\n',
 '<think>\nOkay, let me try to figure out what\'s going on here. The user wants to extract links from a PDF in their downloads folder. The assistant first tried to list the directory, but the response was just "[]", which probably means there were no files listed. Wait, maybe the path is incorrect or the PDF isn\'t there? The user might have a typo in the folder name or the PDF isn\'t actually in that directory.\n\nHmm, the next step should be to check if the PDF exists. Since listing the directory returned an empty array, maybe the path is wrong. The assistant should probably ask

In [9]:
from radarange_orchestrator.utils.display import display_message


display_message(inner_chat[-1])

In [10]:
inner_chat.display_thoughts()